In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import os
from analysis import info_datasets, unique_datasets_list, add_paths
from datasets import DatasetFactory

root_dataset = os.path.join('/mnt', 'data', 'turtles', 'datasets', 'datasets')
root_dataframe = os.path.join('/mnt', 'data', 'turtles', 'datasets', 'dataframes')

info_datasets = add_paths(info_datasets, root_dataset, root_dataframe)

datasets = []
for info_dataset in info_datasets:
    dataset = info_dataset[0].from_file(info_dataset[2], info_dataset[3], **info_dataset[1])
    datasets.append(dataset)

In [3]:
import numpy as np
from splits import Lcg, Split

#df = methods[6].df
df = datasets[2].df

n = 100
n_train = 80
seed = 666
split = Split(df, seed)
df_train1, df_test1 = split.random_split(0.8)
df_train2, df_test2 = split.closed_set_split(0.8)
df_train3, df_test3 = split.semiopen_set_split(0.8, 0.05)
df_train4, df_test4 = split.open_set_split(0.05)

df_trains = [df_train1, df_train2, df_train3, df_train4]
df_tests = [df_test1, df_test2, df_test3, df_test4]

In [4]:
# Number of ignored (unknown) photos
for df_train, df_test in zip(df_trains, df_tests):
    print(len(df) - len(df_train) - len(df_test))

0
0
0
0


In [5]:
# Ratio of train/test size
for df_train, df_test in zip(df_trains, df_tests):
    print(len(df_train) / len(df_test))

4.0
4.037209302325581
3.967889908256881
19.433962264150942


In [6]:
# Ratio of individuals only in the test size
for df_train, df_test in zip(df_trains, df_tests):
    df_train_id = df_train['identity'].unique()
    df_test_id = df_test['identity'].unique()
    n_test_only = 0
    for identity in df_test_id:
        if identity not in df_train_id:
            n_test_only += sum(df['identity'] == identity)
    print(n_test_only / (len(df_train)+len(df_test)))

0.0
0.0
0.041735918744228995
0.048938134810710986


In [7]:
# Ratio of train/test size within all individials
identities = df['identity'].unique()
identities = np.setdiff1d(identities, 'unknown')

for df_train, df_test in zip(df_trains, df_tests):
    ratio = np.array([sum(df_train.identity == k) for k in identities]) / np.array([sum(df_test.identity == k) for k in identities])
    print(ratio)
    failed_counts = []
    for k in np.where(ratio == np.inf)[0]:
        failed_counts.append(sum(df.identity == identities[k]))
    print(np.array(failed_counts))

[ 3.                 inf  3.          5.5         6.          3.8
  2.14285714  6.2         4.5         6.          4.6         4.5
  4.81818182  5.          2.25       13.         10.                 inf
  4.         10.          5.5         3.4         4.5         4.
  3.         21.          3.          2.5         9.          1.8
  2.          3.4         3.5         9.          4.          4.
  8.5         2.14285714  1.6         1.2         3.57142857  2.6
  8.          2.8         2.33333333  3.          3.         12.
  3.14285714  1.44444444         inf  2.25        5.         10.
  3.          6.33333333 10.          3.66666667  4.5                inf
 10.         23.          4.          2.          2.14285714  4.71428571
  1.33333333         inf  6.5         5.8         7.5         6.
  9.          4.          6.          2.5         2.66666667  1.75
  2.66666667  3.8         1.6                inf  5.         11.
  7.66666667  9.          2.66666667  2.33333333  5.4       

/mnt/appl/software/SciPy-bundle/2020.11-fosscuda-2020b/lib/python3.8/site-packages/numpy/lib/arraysetops.py:576: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
<ipython-input-7-d6379ad901df>:6: RuntimeWarning: divide by zero encountered in true_divide
  ratio = np.array([sum(df_train.identity == k) for k in identities]) / np.array([sum(df_test.identity == k) for k in identities])
<ipython-input-7-d6379ad901df>:6: RuntimeWarning: divide by zero encountered in true_divide
  ratio = np.array([sum(df_train.identity == k) for k in identities]) / np.array([sum(df_test.identity == k) for k in identities])
